In [7]:
import os
import pandas as pd

dir = os.getcwd()

cities = pd.read_excel(os.path.join(dir, 'Data', 'municipios_seleccionados.xlsx'))
stations = pd.read_csv(os.path.join(dir, 'Data', 'Catálogo_Nacional_de_Estaciones_del_IDEAM_20251024.csv'))

In [15]:
# Unir por Municipio
stations_selected = pd.merge(
    cities[['municipio']],                # municipios seleccionados
    stations,
    left_on='municipio',
    right_on='Municipio',
    how='inner'                           # solo coincidencias exactas
)
stations_selected = stations_selected[['Departamento','Municipio','Codigo','Nombre','Categoria','Tecnologia','LONGITUD','LATITUD','Altitud','Fecha_instalacion','Fecha_suspension']]

In [16]:
# Seleccionar columnas solicitadas y renombrar si se desea coherencia

stations_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Departamento       172 non-null    object 
 1   Municipio          172 non-null    object 
 2   Codigo             172 non-null    int64  
 3   Nombre             172 non-null    object 
 4   Categoria          172 non-null    object 
 5   Tecnologia         172 non-null    object 
 6   LONGITUD           172 non-null    float64
 7   LATITUD            172 non-null    float64
 8   Altitud            172 non-null    object 
 9   Fecha_instalacion  171 non-null    object 
 10  Fecha_suspension   55 non-null     object 
dtypes: float64(2), int64(1), object(8)
memory usage: 14.9+ KB
